# 1. 라이브러리 설치

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import warnings
warnings.filterwarnings(action='ignore')

# 2. 사용자 설정

2-1. 파일 경로, 파일명, 설정값 설정

In [ ]:
##### 파일명 설정 #####
raw_file = '운행 데이터_위경도좌표.csv'
calendar_file = '달력 데이터.xlsx'

##### 내부/외부 이동 시간 정상값 최대치 설정 #####
in_time_limit = 180
out_time_limit = 360

##### 내부/외부 이동 거리 정상값 최대치 설정 #####
in_dist_limit = 50
out_dist_limit = 150

##### 이상값 삭제 or 유지 옵션 설정 #####
date_del_opt = '삭제'     # '유지'
dist_del_opt = '삭제'     # '유지'

# 3. 데이터 불러오기

3-1. 파일 경로 설정

In [ ]:
default_dir = 'C:/19Project/05.TRV/01.Pred_Demand/'
data_dir = default_dir + '0. 데이터/input/'

3-2. 디렉터리 생성

In [ ]:
now = datetime.datetime.now().date().strftime('%Y%m%d')

num = 1

data_dir_output = default_dir + '0. 데이터/output_' + now
preprocessing_dir = default_dir + '1. 전처리_' + now

while os.path.exists(data_dir_output + '_' + str(num)):
    num += 1

data_dir_output_num = data_dir_output + '_' + str(num) + '/'
preprocessing_dir_num = preprocessing_dir + '_' + str(num) + '/'
    
os.makedirs(os.path.join(data_dir_output_num))
os.makedirs(os.path.join(preprocessing_dir_num))

3-3. 운행 데이터, 달력 데이터 불러오기 (데이터 컬럼 순서 확인)

In [ ]:
data = pd.read_csv(data_dir + raw_file, engine = 'python', encoding='utf-8',converters = {1: str}, 
                         parse_dates = [3,6,10])

In [ ]:
str(data['콜접수 시간'])
import datetime
str_datetime = data['콜접수 시간']
format = '%H:%M'
dt_datetime = datetime.datetime.strptime(str_datetime,format)

In [ ]:
type(data['콜접수 시간'])

pandas.core.series.Series

In [ ]:
data['콜접수 시각'] = arr.iloc[0]  

In [ ]:
data = pd.read_csv(data_dir + raw_file, engine = 'python', encoding='cp949',converters = {1: str, 4: int, 5: int, 8: int, 9: int, 12: int, 13: int}, 
                         parse_dates = [3,7,11])
data.columns = ['seq', '차량 번호', '차량 소속','콜접수 일자', '콜접수 시각', '콜접수 분', '장애 유형', '승차 일자', '승차 시각', 
                    '승차 분', '승차 구역명', '하차 일자', '하차 시각', '하차 분', '하차 구역명', '운행 거리', '픽업 거리']
data.sort_values('seq', inplace = True)

calendar_data = pd.read_excel(data_dir + calendar_file, converters = {1: int}, parse_dates = [0])
calendar_data.columns = ['일자', '일자 특성']

# 4. 함수 생성

4-1. 컬럼 생성 함수

In [ ]:
# 일자 특성 구분 생성 함수
def f_calendar(x):
    if x in [1, 4, 6] : return '평일'
    elif x in [2, 3, 5, 7]: return '휴일'
    
    
# 일시 컬럼 생성 함수 
def datetime_col(df, col):   
    df[col + ' 일시'] = [datetime.datetime.combine(df.loc[i, col + ' 일자'], 
                         datetime.time(df.loc[i, col + ' 시각'], df.loc[i, col+ ' 분']))
                         for i in df.index]
    

# 요일 컬럼 생성 함수
def weekday_col(df, col):   
    weekday_mapping = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
    df[col + ' 요일코드'] = [d.date().weekday() if d.date() != datetime.date(1900, 1, 1) else 999 for d in df[col + ' 일시']]
    df[col + ' 요일'] = df[col + ' 요일코드'].apply(lambda x: weekday_mapping.get(x))
    del df[col + ' 요일코드']   
    
    
# 일자 특성 컬럼 생성
def daycode_col(df, col):
    calendar_data.columns = [col + ' 일자', col + ' 일자 특성']
    tmp = df.merge(calendar_data, on = col + ' 일자', how = 'left')
    df[col + ' 일자 특성'] = tmp[col + ' 일자 특성']
    
    
# 일자 특성 구분 컬럼 생성    
def dayclass_col(df, col):    
    df[col + ' 일자 특성 구분'] = df[col + ' 일자 특성'].apply(lambda x: f_calendar(x))
    

# 이동 구분 컬럼 생성
def move_col(df):
    df['이동 구분'] = df['승차 구역명'] == df['하차 구역명']
    df['이동 구분'] = df['이동 구분'] * 1
    df['이동 구분'] = df['이동 구분'].replace(1, '내부 이동')
    df['이동 구분'] = df['이동 구분'].replace(0, '외부 이동')
    
    
# 콜접수 일시 컬럼 생성 함수
def call_datetime(df):
    df['콜접수 일시'] = datetime.datetime(1900, 1, 1, 0, 0, 0)
    tmp = df[(df['콜접수 일자'].notnull()) & (df['콜접수 시각'].notnull()) & (df['콜접수 분'].notnull())]
    if len(tmp) > 0:
        for i in tmp.index:
            df.loc[i, '콜접수 일시'] = datetime.datetime.combine(df.loc[i, '콜접수 일자'], 
                                       datetime.time(int(df.loc[i, '콜접수 시각']), int(df.loc[i, '콜접수 분'])))


# 연월 컬럼 생성 함수
def ym_col(df, col): 
    df[col + ' 연월'] = [dt. strftime('%Y-%m') for dt in data[col + ' 일시']]
    
    
# 승차 대기 시간 생성 함수
def waiting_time(df):
    df['대기 시간'] = df['승차 일시'] - df['콜접수 일시']
    df['대기 시간'] = df['대기 시간'].apply(lambda x: x.total_seconds() / 60)    
    df['대기 시간'] = [df.loc[i, '대기 시간'] if df.loc[i, '콜접수 일시'] != datetime.datetime(1900, 1, 1, 0, 0, 0) else np.nan 
                       for i in df.index]

4-2. 전처리 함수

In [ ]:
# 날짜 데이터타입 확인 함수 (에러 발생시 입력 데이터 검토 필요)   
def preprocessing_0(df, c_df): # assert 식이 거짓이면 오류 발생    
    date_type = "<class 'pandas._libs.tslibs.timestamps.Timestamp'>"
    test_call = df[df['콜접수 일자'].notnull()]
    test_geton = df[df['승차 일자'].notnull()]
    test_getoff = df[df['하차 일자'].notnull()]
    test_calendar = c_df[c_df['일자'].notnull()]
    if len(test_call) > 0:
        assert len(test_call[test_call['콜접수 일자'].apply(lambda x: str(type(x))) != date_type]) == 0, '[콜접수 일자] 날짜 데이터 타입 오류'
    assert len(test_geton[test_geton['승차 일자'].apply(lambda x: str(type(x))) != date_type]) == 0, '[승차 일자] 날짜 데이터 타입 오류'
    assert len(test_getoff[test_getoff['하차 일자'].apply(lambda x: str(type(x))) != date_type]) == 0, '[하차 일자] 날짜 데이터 타입 오류'    
    assert len(test_calendar[test_calendar['일자'].apply(lambda x: str(type(x))) != date_type]) == 0, '[달력 데이터 일자] 날짜 데이터 타입 오류'
    
    
# null, 유일값 확인 함수 (에러 발생시 입력 데이터 검토 필요)
def preprocessing_1(df, col): # assert 식이 거짓이면 오류 발생    
    assert len(df[df[col].isnull()]) == 0 , '[' + col + '] null값 존재'   
    assert len(df[col].unique()) == len(df), '[' + col + '] 중복값 존재'
    
    
# null 값인 경우 '불명'으로 변경 함수
def preprocessing_2(df, col): 
    tmp = df[df[col].isnull()]    
    if len(tmp) > 0:
        tmp_name = '전처리_' + col + '_null.csv'
        tmp.to_csv(preprocessing_dir_num + tmp_name, encoding = 'MS949', index = False)
        print('\n[' + col + '] null 존재 - "불명"으로 대체 \n' + preprocessing_dir_num + tmp_name)
        df[col] = df[col].fillna('불명')
    
    
# null 값이면 해당 행 삭제 함수
def preprocessing_3(df, col):    
    tmp = df[df[col].isnull()]    
    if len(tmp) > 0:
        tmp_name = '전처리_' + col + '_null.csv'
        tmp.to_csv(preprocessing_dir_num + tmp_name, encoding = 'MS949', index = False)
        print('\n[' + col +'] null 존재 - 내역 삭제 \n' + preprocessing_dir_num + tmp_name)        
        df.dropna(axis = 0, subset = [col], inplace = True)
        df.reset_index(drop = True, inplace = True)
    
    
# 시각, 분 검증 함수 (null값, 이상값 -> 해당 행 삭제)
def preprocessing_4(df, col):
    tmp_1 = df[df[col].isnull()]   
    if len(tmp_1) > 0:
        tmp_1_name = '전처리_' + col + '_null.csv'
        tmp_1.to_csv(preprocessing_dir_num + tmp_1_name, encoding = 'MS949', index = False)
        print('\n[' + col +'] null 존재 - 내역 삭제 \n' + preprocessing_dir_num + tmp_1_name)      
        df.dropna(axis = 0, subset = [col], inplace = True)
        df.reset_index(drop = True, inplace = True)
    if '시각' in col:
        end_value = 23
    elif '분' in col:
        end_value = 59        
    tmp_2 = df[((df[col] < 0) | (df[col] > end_value))]    
    if len(tmp_2) > 0:
        tmp_2_name = '전처리_' + col + '_이상값.csv'
        tmp_2.to_csv(preprocessing_dir_num + tmp_2_name, encoding = 'MS949', index = False)
        print('\n[' + col +'] 이상값 존재 - 내역 삭제 \n' + preprocessing_dir_num + tmp_2_name)
        df.drop(index = tmp_2.index, inplace = True)
        df.reset_index(drop = True, inplace = True)


# 운행데이터 기간 범위 검증 함수 (기간 범위 3개년도 제한)
def preprocessing_5(df):    
    start_y = df['승차 일시'].max().year - 2
    tmp = df[df['승차 일시'].apply(lambda x: x.year) < start_y]
    if len(tmp) > 0:
        tmp_name = '전처리_승차 일시_기간 범위 외.csv'
        tmp.to_csv(preprocessing_dir_num + tmp_name, encoding = 'MS949', index = False)
        print('\n[승차 일시] 기간 범위 외 존재 - 내역 삭제 \n' + preprocessing_dir_num + tmp_name)    
        df.drop(index = tmp.index, inplace = True)
        df.reset_index(drop = True, inplace = True)


# 하차 일시 검증 함수 (승차 일시와 비교)
def preprocessing_6(df, del_option = '유지', in_time = 180, out_time = 360):  
    # 이동 시간 디폴트값 (내부: 180분, 외부: 360분) / del_option에 '삭제' 입력 시 행 삭제
    # 승차 일시와 비교 후 삭제
    df['diff'] = df['하차 일시'] - df['승차 일시']
    df['diff'] = df['diff'].apply(lambda x: x.total_seconds() / 60)    
    tmp_1 = df[df['diff'] < 0]    
    if len(tmp_1) > 0:
        tmp_1_name = '전처리_하차 일시_승차 일시 이전.csv'
        tmp_1.to_csv(preprocessing_dir_num + tmp_1_name, encoding = 'MS949', index = False)
        print('\n[하차 일시] 승차일시 이전 값 존재 - 내역 삭제 \n' + preprocessing_dir_num + tmp_1_name)
        df.drop(index = tmp_1.index, inplace = True)
        df.reset_index(drop = True, inplace = True)
    # 승차 일시와 차이값
    df['time_value'] = df['이동 구분']
    df['time_value'] = df['time_value'].replace('내부 이동', in_time)
    df['time_value'] = df['time_value'].replace('외부 이동', out_time)
    tmp_2 = df[df['diff'] >= df['time_value']]
    if len(tmp_2) > 0:
        tmp_2_name = '전처리_하차 일시_승차 일시와 차이값 이상.csv'
        tmp_2.to_csv(preprocessing_dir_num + tmp_2_name, encoding = 'MS949', index = False)        
        if del_option == '삭제':
            df.drop(index = tmp_2.index, inplace = True)
            df.reset_index(drop = True, inplace = True)
            print('\n[하차 일시] 승차일시와 차이 이상값 - 내역 삭제 \n' + preprocessing_dir_num + tmp_2_name)
        else :
            print('\n[하차 일시] 승차일시와 차이 이상값 - 검토 필요  \n' + preprocessing_dir_num + tmp_2_name)
    del df['diff']
    del df['time_value']   


# 거리 이상값 검토 함수
def preprocessing_7(df, del_option = '유지', in_distance = 50, out_distance = 150):
    # 거리 디폴트값 (내부: 50, 외부: 150) / del_option에 '삭제' 입력 시 행 삭제
    df['distance_value'] = df['이동 구분']
    df['distance_value'] = df['distance_value'].replace('내부 이동', in_distance)
    df['distance_value'] = df['distance_value'].replace('외부 이동', out_distance)    
    tmp = df[df['운행 거리'] >= df['distance_value']]
    if len(tmp) > 0:
        tmp_name = '전처리_운행 거리_이상값.csv'
        tmp.to_csv(preprocessing_dir_num + tmp_name, encoding = 'MS949', index = False) 
        if del_option == '삭제':
            df.drop(index = tmp.index, inplace = True)
            df.reset_index(drop = True, inplace = True)
            print('\n[운행 거리] 이상값 - 내역 삭제 \n' + preprocessing_dir_num + tmp_name)
        else:
            print('\n[운행 거리] 이상값 - 검토 필요 \n' + preprocessing_dir_num + tmp_name)            
    del df['distance_value']
    
    
# 거리 값이 null, 0값인 경우 중앙값 대체 함수 (0을 제외하고 계산한 이동 구분 각각의 중앙값)
def preprocessing_8(df):   
    in_median = round(df[(df['이동 구분'] == '내부 이동') & (df['운행 거리'] > 0)]['운행 거리'].median(), 2)
    out_median = round(df[(df['이동 구분'] == '외부 이동') & (df['운행 거리'] > 0)]['운행 거리'].median(), 2)   
    # 거리 0값 리스트 저장    
    tmp = df[(df['운행 거리'] == 0) | (df['운행 거리'].isnull())]
    if len(tmp) > 0:
        tmp_name = '전처리_운행 거리_0km 또는 null.csv'
        tmp.to_csv(preprocessing_dir_num + tmp_name, encoding = 'MS949', index = False)
        print('\n[운행 거리] 0km 또는 null - 중앙값 대체 \n' + preprocessing_dir_num + tmp_name)   
        # 변경
        df.loc[(df['이동 구분'] == '내부 이동') & ((df['운행 거리'] == 0) | (df['운행 거리'].isnull())), '운행 거리'] = in_median
        df.loc[(df['이동 구분'] == '외부 이동') & ((df['운행 거리'] == 0) | (df['운행 거리'].isnull())), '운행 거리'] = out_median
    
    
# 콜접수 일자, 시각, 분 null값 처리 함수 (1900-01-01 00:00:00 처리)
def preprocessing_9(df): # 콜접수 일자 / 콜접수 시각/ 콜접수 분
    col_list = ['콜접수 일자', '콜접수 시각', '콜접수 분']
    for c in col_list:
        tmp = df[df[c].isnull()]
        if len(tmp) > 0:
            tmp_name = '전처리_' + c + '_null값.csv'
            tmp.to_csv(preprocessing_dir_num + tmp_name, encoding = 'MS949', index = False)
            print('\n[' + c +'] null 존재 - 1900-01-01 처리 \n' + preprocessing_dir_num + tmp_name)
            df.loc[tmp.index, '콜접수 일자'] = datetime.date(1900, 1, 1)
            df.loc[tmp.index, ['콜접수 시각', '콜접수 분']] = 0        

        
# 콜접수 시각, 분 이상값 검증 함수 (시각, 분이 이상값인 경우 해당 행 1900-01-01 00:00:00 처리)
def preprocessing_10(df, col):
    if col == '콜접수 시각':
        end_value = 23
    elif col == '콜접수 분':
        end_value = 59
    tmp = df[((df[col] < 0) | (df[col] > end_value))]
    if len(tmp) > 0:
        tmp_name = '전처리_' + col + '_이상값.csv'
        tmp.to_csv(preprocessing_dir_num + tmp_name, encoding = 'MS949', index = False)
        print('\n[' + col +'] 이상값 - 1900-01-01 처리 \n' + preprocessing_dir_num + tmp_name)
        df.loc[tmp.index, '콜접수 일자'] = datetime.date(1900, 1, 1)
        df.loc[tmp.index, ['콜접수 시각', '콜접수 분']] = 0

        
# 콜접수 일시 검증 함수 (승차일자와 비교)
def preprocessing_11(df):
    call_data = df[df['콜접수 일시'] != datetime.datetime(1900, 1, 1, 0, 0, 0)]
    if len(call_data) > 0:
        #승차일자 전날보다 이전인 경우
        boarding_1day_before = call_data['승차 일자'] - datetime.timedelta(days = 1)
        tmp_1 = call_data[call_data['콜접수 일자'] < boarding_1day_before]
        if len(tmp_1) > 0 :
            tmp_1_name = '전처리_콜접수 일시_승차 일자 차이 2일 이상.csv'
            tmp_1.to_csv(preprocessing_dir_num + tmp_1_name, encoding = 'MS949', index = False)
            print('\n[콜접수 일시] 승차 일자 차이 2일 이상 - 1900-01-01 처리 \n' + preprocessing_dir_num + tmp_1_name)
            df.loc[tmp_1.index, '콜접수 일시'] = datetime.datetime(1900, 1, 1, 0, 0, 0)
            df.loc[tmp_1.index, '콜접수 일자'] = datetime.date(1900, 1, 1)
            df.loc[tmp_1.index, ['콜접수 시각', '콜접수 분']] = 0
            df.loc[tmp_1.index, ['콜접수 요일', '콜접수 일자 특성', '콜접수 일자 특성 구분']] = np.nan
        #승차일자 이후인 경우
        call_data['diff'] = call_data['콜접수 일시'] - call_data['승차 일시']
        call_data['diff'] = call_data['diff'].apply(lambda x: x.total_seconds() / 60)
        tmp_2 = call_data[call_data['diff'] > 0]
        if len(tmp_2) > 0 :
            tmp_2_name = '전처리_콜접수 일시_승차 일시 이후.csv'
            tmp_2.to_csv(preprocessing_dir_num + tmp_2_name, encoding = 'MS949', index = False)
            print('\n[콜접수 일시] 승차 일시 이후 - 1900-01-01 처리 \n' + preprocessing_dir_num + tmp_2_name)
            df.loc[tmp_2.index, '콜접수 일시'] = datetime.datetime(1900, 1, 1, 0, 0, 0)
            df.loc[tmp_2.index, '콜접수 일자'] = datetime.date(1900, 1, 1)
            df.loc[tmp_2.index, ['콜접수 시각', '콜접수 분']] = 0
            df.loc[tmp_2.index, ['콜접수 요일', '콜접수 일자 특성', '콜접수 일자 특성 구분']] = np.nan
            
            
# 일자 특성 검증 함수
def preprocessing_12(df):
    ymd_1 = df[['승차 일자', '승차 일자 특성']]
    ymd_2 = df[['콜접수 일자', '콜접수 일자 특성']]
    ymd_2 = ymd_2[ymd_2['콜접수 일자'].notnull()]
    ymd_1.columns = ['일자', '일자 특성']
    ymd_2.columns = ['일자', '일자 특성']
    ymd_table = pd.concat([ymd_1, ymd_2])
    ymd_table = ymd_table.drop_duplicates(keep='first')
    ymd_table = ymd_table[ymd_table['일자'] != '1900-01-01']
    # 일자특성 null
    tmp_1 = ymd_table[ymd_table['일자 특성'].isnull()]
    if len(tmp_1) > 0:
        ymd_list1 = pd.Series(tmp_1['일자'].unique())
        print('\n[일자 특성 null] 달력 데이터 검토 필요')
        for i in ymd_list1:
            print(i)
    # 일자특성 오류값
    tmp_2 = ymd_table[(ymd_table['일자 특성'].notnull()) & (~ ymd_table['일자 특성'].isin(range(8)))]
    if len(tmp_2) > 0:
        ymd_list2 = pd.Series(tmp_2['일자'].unique())
        print('\n[일자 특성 이상값] 달력 데이터 검토 필요')
        for j in ymd_list2:
            print(j)

4-3. 집계 테이블 생성 함수

In [ ]:
def add_table(df):
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    for c in car_type:
        if c != '전체':
            tmp = df[df['차량 소속'] == c]
        elif c == '전체':
            tmp = df
        # 집계
        tmp['승차 분 구분'] = [0 if m < 30 else 30 for m in tmp['승차 분']]
        tmp_table = pd.DataFrame(tmp.groupby(['승차 일자', '승차 요일', '승차 일자 특성 구분', '승차 시각', '승차 분 구분', '승차 구역명', '하차 구역명'])['seq'].count())
        tmp_table.reset_index(inplace = True)
        tmp_table.sort_values(['승차 일자', '승차 시각', '승차 분 구분', '승차 구역명', '하차 구역명'], inplace = True)
        tmp_table.reset_index(drop = True, inplace = True)
        tmp_table.columns = ['승차 일자', '승차 요일', '승차 일자 특성 구분', '승차 시각', '승차 분', '승차 구역명', '하차 구역명', '운행 수']
        # 파일 저장
        tmp_table.to_csv(data_dir_output_num + '분석테이블_집계_' + c + '.csv', index = False, encoding = 'MS949')

# 5. 데이터 전처리

5-1. 날짜 데이터 타입 검증 (콜접수 일자, 승차 일자, 하차 일자, 달력데이터 일자) -> 에러 발생시 해당 내역 검토

In [ ]:
preprocessing_0(data, calendar_data)

5-2. 운행데이터 seq, 달력데이터 일자 검증 (null, 중복값 검토) -> 에러 발생시 해당 내역 검토

In [ ]:
preprocessing_1(data, 'seq')
preprocessing_1(calendar_data, '일자')

5-3. 차량번호, 차량 소속, 장애유형 전처리 (null -> '불명')

In [ ]:
preprocessing_2(data, '차량 번호')
preprocessing_2(data, '차량 소속')
preprocessing_2(data, '장애 유형')


[차량 번호] null 존재 - "불명"으로 대체 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_차량 번호_null.csv

[장애 유형] null 존재 - "불명"으로 대체 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_장애 유형_null.csv


5-4. 승하차 일자, 승하차 구역명 전처리 (null -> 행 삭제)

In [ ]:
preprocessing_3(data, '승차 일자')
preprocessing_3(data, '하차 일자')
preprocessing_3(data, '승차 구역명')
preprocessing_3(data, '하차 구역명')


[승차 일자] null 존재 - 내역 삭제 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_승차 일자_null.csv


5-5. 승하차 시각, 분 전처리 (null, 이상값 -> 행 삭제)

In [ ]:
preprocessing_4(data, '승차 시각')
preprocessing_4(data, '승차 분')
preprocessing_4(data, '하차 시각')
preprocessing_4(data, '하차 분')


[하차 시각] null 존재 - 내역 삭제 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_하차 시각_null.csv

[하차 시각] 이상값 존재 - 내역 삭제 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_하차 시각_이상값.csv

[하차 분] null 존재 - 내역 삭제 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_하차 분_null.csv

[하차 분] 이상값 존재 - 내역 삭제 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_하차 분_이상값.csv


5-6. 승하차 일시 컬럼 생성

In [ ]:
datetime_col(data, '승차')
datetime_col(data, '하차')

5-7. 승차 요일, 일자 특성, 일자 특성 구분 컬럼 생성

In [ ]:
weekday_col(data, '승차')
daycode_col(data, '승차')
dayclass_col(data, '승차')

5-8. 이동 구분 컬럼 생성

In [ ]:
move_col(data)

5-9. 승차 일시 기간 범위 전처리 (기간 범위 3개년도로 제한)

In [ ]:
preprocessing_5(data)


[승차 일시] 기간 범위 외 존재 - 내역 삭제 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_승차 일시_기간 범위 외.csv


5-10. 하차 일시 전처리 (하차 일시가 승차 일시 이전인 경우 -> 삭제 / 승차 일시와 일정 시간 이상 차이 -> 검토 또는 삭제)

In [ ]:
preprocessing_6(data, del_option = date_del_opt, in_time = in_time_limit, out_time = out_time_limit)


[하차 일시] 승차일시 이전 값 존재 - 내역 삭제 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_하차 일시_승차 일시 이전.csv

[하차 일시] 승차일시와 차이 이상값 - 내역 삭제 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_하차 일시_승차 일시와 차이값 이상.csv


5-11. 운행 거리 전처리1 (일정 거리 이상 -> 검토 또는 삭제)

In [ ]:
preprocessing_7(data, del_option = dist_del_opt, in_distance = in_dist_limit, out_distance = out_dist_limit)


[운행 거리] 이상값 - 내역 삭제 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_운행 거리_이상값.csv


5-12. 운행 거리 전처리2 (0, null -> 내부 외부 각각 중앙값 대체) : 운행 거리 전처리1 후 실행

In [ ]:
preprocessing_8(data)


[운행 거리] 0km 또는 null - 중앙값 대체 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_운행 거리_0km 또는 null.csv


5-13. 콜접수 일자, 시각, 분 검증 (null -> 1900-01-01 00:00:00)

In [ ]:
preprocessing_9(data)


[콜접수 일자] null 존재 - 1900-01-01 처리 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_콜접수 일자_null값.csv

[콜접수 시각] null 존재 - 1900-01-01 처리 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_콜접수 시각_null값.csv

[콜접수 분] null 존재 - 1900-01-01 처리 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_콜접수 분_null값.csv


5-14. 콜접수 시각, 분 전처리 (이상값 -> 1900-01-01 00:00:00)

In [ ]:
preprocessing_10(data, '콜접수 시각')
preprocessing_10(data, '콜접수 분')


[콜접수 시각] 이상값 - 1900-01-01 처리 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_콜접수 시각_이상값.csv

[콜접수 분] 이상값 - 1900-01-01 처리 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_콜접수 분_이상값.csv


5-15. 콜접수 일시, 요일, 일자 특성, 일자 특성 구분 컬럼 생성

In [ ]:
data['콜접수 일자']  = pd.to_datetime(data['콜접수 일자'], format = "%Y-%m-%d", errors = 'raise')

In [ ]:
datetime_col(data, '콜접수')
weekday_col(data, '콜접수')
daycode_col(data, '콜접수')
dayclass_col(data, '콜접수')

5-16. 콜접수 일시 전처리 (승차 일자와 차이가 2일 이상, 승차 일자 이후 -> 1900-01-01 00:00:00)

In [ ]:
preprocessing_11(data)


[콜접수 일시] 승차 일시 이후 - 1900-01-01 처리 
C:/19Project/05.TRV/01.Pred_Demand/1. 전처리_20211006_1/전처리_콜접수 일시_승차 일시 이후.csv


5-17. 달력 데이터 검증 (일자 특성 검증)

In [ ]:
preprocessing_12(data)


[일자 특성 null] 달력 데이터 검토 필요
1900-01-01 00:00:00


5-18. 승차, 콜접수 연월 컬럼 생성

In [ ]:
ym_col(data, '승차')
ym_col(data, '콜접수')

5-19. 승차 대기 시간 추가

In [ ]:
waiting_time(data)

5-20. 일자 데이터 타입 변경

In [ ]:
data['승차 일자'] = data['승차 일자'].apply(lambda x: x.date())
data['하차 일자'] = data['하차 일자'].apply(lambda x: x.date())
#data['콜접수 일자'] = data['콜접수 일자'].apply(lambda x: x.date())

# 6. 데이터 저장

6-1. 데이터 컬럼 순서 지정

In [ ]:
data = data[['seq', '차량 번호', '차량 소속', '콜접수 일자', '콜접수 시각', '콜접수 분', '장애 유형', '승차 일자', '승차 시각', 
             '승차 분', '승차 구역명', '하차 일자', '하차 시각', '하차 분', '하차 구역명', '운행 거리', '픽업 거리', '이동 구분', 
             '콜접수 일시', '콜접수 연월', '콜접수 요일', '콜접수 일자 특성', '콜접수 일자 특성 구분', '승차 일시', '승차 연월',
             '승차 요일', '승차 일자 특성', '승차 일자 특성 구분', '하차 일시', '대기 시간']]

6-2. 전처리 완료 데이터 저장

In [ ]:
data.to_csv(data_dir_output_num + '운행데이터_전처리 완료.csv', index = False, encoding = 'MS949')

6-3. 구역 리스트 저장

In [ ]:
addr_list = pd.DataFrame({'구역명' : pd.concat([data['승차 구역명'], data['하차 구역명']]).unique()})
addr_list.sort_values('구역명', inplace = True)
addr_list.to_csv(data_dir_output_num + 'tmp_addr_list.csv', index = False, encoding = 'MS949')

6-4. 승차 구역+하차 구역 리스트 저장

In [ ]:
od_list = data[['승차 구역명', '하차 구역명']]
od_list = od_list.drop_duplicates(keep = 'first')
od_list.sort_values(['승차 구역명', '하차 구역명'], inplace = True)
od_list.reset_index(drop = True, inplace = True)
od_list.to_csv(data_dir_output_num + 'tmp_zone_list.csv', index = False, encoding = 'MS949')

6-5. 시간 리스트 저장

In [ ]:
time_list = pd.DataFrame({'운행 시간' : data['승차 시각'].unique()})
time_list.sort_values('운행 시간', inplace = True)
time_list.reset_index(drop = True, inplace = True)
time_list.to_csv(data_dir_output_num + 'tmp_time_list.csv', index = False, encoding = 'MS949')

6-6. 집계 테이블 저장

In [ ]:
add_table(data)